In [1]:
%%shell
#setup env
git clone https://github.com/tensorflow/privacy
cd privacy
pip install -e .

Cloning into 'privacy'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 636 (delta 32), reused 54 (delta 24), pack-reused 568
Receiving objects: 100% (636/636), 264.10 KiB | 983.00 KiB/s, done.
Resolving deltas: 100% (362/362), done.
Obtaining file:///content/privacy
  Running setup.py develop for privacy


In [0]:
!pip list

In [2]:
import privacy



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
!pwd

/content


In [0]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('./train.csv')
train_labels = pd.read_csv('./clean_y.csv')
test_data = pd.read_csv('test_x.csv')

train_data = np.array(train_data)
test_data = np.array(test_data)
train_labels = np.array(train_labels)
train_data = train_data[:100000,1:]
train_labels = train_labels[:100000,2]
test_data = test_data[:100000,1:]

In [77]:
train_labels

array([1., 1., 0., ..., 1., 1., 0.])

In [0]:
# Copyright 2019, The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Training a CNN on MNIST with Keras and the DP SGD optimizer."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.dp_query.gaussian_query import GaussianAverageQuery
from privacy.optimizers.dp_optimizer import DPGradientDescentOptimizer

# Compatibility with tf 1 and 2 APIs
try:
  GradientDescentOptimizer = tf.train.GradientDescentOptimizer
except:  # pylint: disable=bare-except
  GradientDescentOptimizer = tf.optimizers.SGD  # pylint: disable=invalid-name

#tf.flags.DEFINE_boolean('dpsgd', True, 'If True, train with DP-SGD. If False, '
#                        'train with vanilla SGD.')
#tf.flags.DEFINE_float('learning_rate', 0.15, 'Learning rate for training')
#tf.flags.DEFINE_float('noise_multiplier', 1.1,
#                      'Ratio of the standard deviation to the clipping norm')
#tf.flags.DEFINE_float('l2_norm_clip', 1.0, 'Clipping norm')
#tf.flags.DEFINE_integer('batch_size', 250, 'Batch size')
#tf.flags.DEFINE_integer('epochs', 60, 'Number of epochs')
# tf.flags.DEFINE_integer('microbatches', 250, 'Number of microbatches '
#                         '(must evenly divide batch_size)')
#tf.flags.DEFINE_string('model_dir', None, 'Model directory')

FLAGS = tf.flags.FLAGS
FLAGS.batch_size = 250
FLAGS.learning_rate = 0.001
def compute_epsilon(steps):
  """Computes epsilon value for given hyperparameters."""
  if FLAGS.noise_multiplier == 0.0:
    return float('inf')
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = FLAGS.batch_size / 100000
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=FLAGS.noise_multiplier,
                    steps=steps,
                    orders=orders)
  # Delta is set to 1e-5 because MNIST has 100000 training points.
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]



def main(unused_argv):
  tf.logging.set_verbosity(tf.logging.INFO)
  if FLAGS.dpsgd and FLAGS.batch_size % FLAGS.microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

  # Define a sequential Keras model
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(64, activation='relu',input_shape=(207,)),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(2,activation = 'softmax')
  ])

  if FLAGS.dpsgd:
    dp_average_query = GaussianAverageQuery(
        FLAGS.l2_norm_clip,
        FLAGS.l2_norm_clip * FLAGS.noise_multiplier,
        FLAGS.microbatches)
    optimizer = DPGradientDescentOptimizer(
        dp_average_query,
        FLAGS.microbatches,
        learning_rate=FLAGS.learning_rate,
        unroll_microbatches=True)
    # Compute vector of per-example loss rather than its mean over a minibatch.
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
  else:
    optimizer = GradientDescentOptimizer(learning_rate=FLAGS.learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

  # Compile model with Keras
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  # Train model with Keras
  model.fit(train_data, train_labels,
            epochs=FLAGS.epochs,
            batch_size=FLAGS.batch_size)
  test_pred = model.predict(test_x)

  # Compute the privacy budget expended.
  if FLAGS.dpsgd:
    eps = compute_epsilon(FLAGS.epochs * 100000 // FLAGS.batch_size)
    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
  else:
    print('Trained with vanilla non-private SGD optimizer')

if __name__ == '__main__':
  tf.app.run()


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
100000/100000 [==============================] - 93s 930us/sample - loss: 0.6948 - acc: 1.0000
Epoch 2/15
100000/100000 [==============================] - 77s 768us/sample - loss: 0.6948 - acc: 1.0000
Epoch 3/15
100000/100000 [==============================] - 79s 785us/sample - loss: 0.6948 - acc: 1.0000
Epoch 4/15
 51250/100000 [==============>...............] - ETA: 38s - loss: 0.6936 - acc: 1.0000

In [0]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('./train.csv')
train_labels = pd.read_csv('./clean_y.csv')
test_data = pd.read_csv('test_x.csv')

train_data = np.array(train_data)
test_data = np.array(test_data)
train_labels = np.array(train_labels)
train_data = train_data[:100000,1:]
train_labels = train_labels[:100000,1]
test_data = test_data[:,1:]

In [0]:
# Copyright 2019, The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Training a CNN on MNIST with Keras and the DP SGD optimizer."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.dp_query.gaussian_query import GaussianAverageQuery
from privacy.optimizers.dp_optimizer import DPGradientDescentOptimizer

# Compatibility with tf 1 and 2 APIs
try:
  GradientDescentOptimizer = tf.train.GradientDescentOptimizer
except:  # pylint: disable=bare-except
  GradientDescentOptimizer = tf.optimizers.SGD  # pylint: disable=invalid-name

#tf.flags.DEFINE_boolean('dpsgd', True, 'If True, train with DP-SGD. If False, '
#                        'train with vanilla SGD.')
#tf.flags.DEFINE_float('learning_rate', 0.15, 'Learning rate for training')
#tf.flags.DEFINE_float('noise_multiplier', 1.1,
#                      'Ratio of the standard deviation to the clipping norm')
#tf.flags.DEFINE_float('l2_norm_clip', 1.0, 'Clipping norm')
#tf.flags.DEFINE_integer('batch_size', 250, 'Batch size')
#tf.flags.DEFINE_integer('epochs', 60, 'Number of epochs')
# tf.flags.DEFINE_integer('microbatches', 250, 'Number of microbatches '
#                         '(must evenly divide batch_size)')
#tf.flags.DEFINE_string('model_dir', None, 'Model directory')

FLAGS = tf.flags.FLAGS
FLAGS.batch_size = 250
def compute_epsilon(steps):
  """Computes epsilon value for given hyperparameters."""
  if FLAGS.noise_multiplier == 0.0:
    return float('inf')
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = FLAGS.batch_size / 100000
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=FLAGS.noise_multiplier,
                    steps=steps,
                    orders=orders)
  # Delta is set to 1e-5 because MNIST has 100000 training points.
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]



def main(unused_argv):
  tf.logging.set_verbosity(tf.logging.INFO)
  if FLAGS.dpsgd and FLAGS.batch_size % FLAGS.microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

  # Define a sequential Keras model
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(64, activation='relu',input_shape=(207,)),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1)
  ])

  if FLAGS.dpsgd:
    dp_average_query = GaussianAverageQuery(
        FLAGS.l2_norm_clip,
        FLAGS.l2_norm_clip * FLAGS.noise_multiplier,
        FLAGS.microbatches)
    optimizer = DPGradientDescentOptimizer(
        dp_average_query,
        FLAGS.microbatches,
        learning_rate=FLAGS.learning_rate,
        unroll_microbatches=True)
    # Compute vector of per-example loss rather than its mean over a minibatch.
    loss = tf.keras.losses.MeanSquaredError(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
  else:
    optimizer = GradientDescentOptimizer(learning_rate=FLAGS.learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

  # Compile model with Keras
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  # Train model with Keras
  model.fit(train_data, train_labels,
            epochs=FLAGS.epochs,
            batch_size=FLAGS.batch_size)
  test_pred = model.predict(test_x)

  # Compute the privacy budget expended.
  if FLAGS.dpsgd:
    eps = compute_epsilon(FLAGS.epochs * 100000 // FLAGS.batch_size)
    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
  else:
    print('Trained with vanilla non-private SGD optimizer')

if __name__ == '__main__':
  tf.app.run()
